## There is a small problem in codes at part 1.2

## To calculate magnetic field of each dipole from distance r,we use 
$\mathbf{B}(\mathbf{r}) = \nabla \times \mathbf{A} = \frac{\mu_0}{4\pi} \left[ \frac{3\mathbf{r}(\mathbf{m} \cdot \mathbf{r})}{r^5} - \frac{\mathbf{m}}{r^3} \right]$


In [12]:
import numpy as np
import math
import plotly.graph_objects as go
#Angle directions uses conventions on that page  --> https://en.wikipedia.org/wiki/Spherical_coordinate_system
#We ignore calculation of H field for reasonable approximation

def coordinate_calculator(radius,number_of_magnets): #Calculates the coordinates in 3D ----->Before Creating HallbachArray object,use this.
    #We will use it as an attribute to feed HalbachArray class 
    coords=[]
    
    
    
    increment=(2*math.pi)/number_of_magnets


    for i in range(number_of_magnets):
        coords.append([radius*np.cos(i*increment),radius*np.sin(i*increment),0])
    return np.array(coords)




class HalbachArray:  #Our array is placed on x-y plane,and the circular gap of array is in z axis.
    full_angle=2*math.pi  #Angle of full circle
    B_strength=1.45 #Tesla// I took values from here --> https://www.stanfordmagnets.com/what-is-the-difference-between-n35-and-n52-magnets.html
    #Dimesions of magnet 10*10*10 mm
    
    def __init__(self,n_of_magnets,radius_of_array,dimensions_of_square_screen,resolution,theta_array,phi_array,coordinates): #Resolution is how much data points we have measured inside of the viewing screen of magnetic field.More precisely resolution is how many points per axis you want to see.
        self.n_of_magnets=n_of_magnets #n_of_magnets is the number of magnets in halbach array 
        self.radius_of_array=radius_of_array
        self.dimensions_of_square_screen=dimensions_of_square_screen #This is the place where we put magnetic bots (For our purpose we will see it as a viewing screen of magnetic field)
        self.resolution=resolution
        self.theta_array=np.deg2rad(np.array(theta_array)) #These are the orientation angles for each magnet we used in hallbach array.From zero'th magnet to nth magnet,we are entering each angle here.
        self.phi_array=np.deg2rad(np.array(phi_array)) #Angles will taken as degrees and converted here to radians for proper calculation
        self.coordinates=coordinates #We will take coordinates as np array from used function above. These are the coordinates of each magnet.
    def is_ok(self): #Check whether input parameters are correct
        if len(self.theta_array)==self.n_of_magnets and len(self.phi_array)==self.n_of_magnets:
            return True
        else:
    
            return False
    
    
   
    
    def coordinates_of_magnets(self): #Each magnet is called with its number.First magnet is numbered as zero.Additionally,first magnet is placed at angle zero in polar coordinates and remaining ones are placed in counter-clockwise direction properly.
        
        coordinates=[]
        angle_increment=(self.full_angle)/self.n_of_magnets
        for i in range(self.n_of_magnets):
            coordinates.append([self.radius_of_array*math.cos(0+i*angle_increment),self.radius_of_array*math.sin(0+i*angle_increment)])
        return np.array(coordinates) #The function returns coordinate arrays at the end
    
    

    
    
    def plot_magnetic_field_screen(self): #It will plot the magnetic field screen with seismic colormap #It will calculate and plot magnetic field via plotly 3D cone plot.
        #We start by defining r unit vector to assign a direction to our magnetic dipole and r_x denotes x component of r.
        m_x=[]
        m_y=[]
        m_z=[]
        for i in range(len(self.theta_array)):
            m_x.append(1*np.sin(self.theta_array[i])*np.cos(self.phi_array[i])) # r=1
            m_y.append(1*np.sin(self.theta_array[i])*np.sin(self.phi_array[i]))
            m_z.append(1*np.cos(self.theta_array[i]))
        #Now define m vector which is valid for each magnet,each 

        #Now we have calculated magnitudes of our direction vector r for our magnetic dipole,we are ready to dive in our magnetic field calculation and plotting
        m_x=np.array(m_x)
        m_y=np.array(m_y)
        m_z=np.array(m_z)



        #Now calculate the s vectors with respect to the origin.(These are places where we take measurement in viewing screen) #Part 1.2
        
        s_coords=[[[-((self.dimensions_of_square_screen)/2)+i*(self.dimensions_of_square_screen/(self.resolution-1)),((self.dimensions_of_square_screen)/2)-j*((self.dimensions_of_square_screen)/(self.resolution-1))    ,0] for j in range(self.resolution)] for i in range(self.resolution)]
                
        



        r_vectors=[] #By starting from 0'th magnet and point [0,0] ,we will calculate r vectors for each point and each magnet. 
        '''Structure [   
                        [
                            [r(00)0,r(00)1,r(00)2,......,r(00)n],
                            [r(01)0,r(01)1,r(01)2,......,r(01)n],                        
                            .....,
                            [r(0n)0,r(0n)1,r(0n)2,......,r(0n)n]
                            
                        ],



                        [
                            [r(10)0,r(10)1,r(10)2,......,r(10)n],
                            [r(11)0,r(11)1,r(11)2,......,r(11)n],
                            .....,
                            [r(1n)0,r(1n)1,r(1n)2,......,r(1n)n],
                        ]
                        .....
                        .....
                        .....
                        .....
                        .....
                        .....
                        [
                            [r(n0)0,r(n0)1,r(n0)2,......,r(n0)n],
                            [r(n1)0,r(n1)1,r(n1)2,......,r(n1)n],
                            .....,
                            [r(nn)0,r(nn)1,r(nn)2,......,r(nn)n]

                        ]
        
        
        
                     ]'''
        #Calculate r_vectors array r(xyn) n is naming number of n'th magnet
        #You need to scan s_coords array with two index
            #---> First index to scan s_coords
             #---> Second index to scan s_coords
                 #n is the index of magnets around the circle
                    #r_vectors.append(self.coordinates[n]-np.array([s_coords[i3][0],s_coords[i3][],s_coords[][]]))
        r_vectors=[[[s_coords[idx1][idx2]-self.coordinates[idx3] for idx1 in range(self.resolution)] for idx2 in range(self.resolution)] for idx3 in range(self.n_of_magnets)     ]
                    

        #Now we have to calculate r vectors for each point on measurement screen
           
           #fill that gap with r calculations for each point

        #Each point in the screen is named with its coordinates [x,y].
        
        def calculate_B_field(m_,r_): # Takes m and r as numpy arrays 
            Mu_naught=4*math.pi*10**(-7)
            B = (Mu_naught/(4*math.pi))*(      (3*r_*(np.dot(m_,r_)))/(np.linalg.norm(r_)**5)     - m_/(np.linalg.norm(r_)**3)                           )
            return B
        





        

## Try to use Plotly

In [13]:
import plotly.graph_objects as go
figure=go.Figure(data=go.Cone(x=[0],y=[0],z=[0],u=[3],v=[3],w=[3],sizemode="absolute",sizeref=2,anchor="tip"))
figure.update_layout(
      scene=dict(domain_x=[0, 1],
                 camera_eye=dict(x=-1.57, y=1.36, z=0.58)))
figure.show()


In [129]:
obj=HalbachArray(4,10,5,6,[90,90,90,90],[90,90,90,90])
obj.resolution





6

In [151]:
def calculate_B_field(m_,r_): # Takes m and r as numpy arrays 
    Mu_naught=4*math.pi*10**(-7)
    B = (Mu_naught/(4*math.pi))*(      (3*r_*(np.dot(m_,r_)))/(np.linalg.norm(r_)**5)     - m_/(np.linalg.norm(r_)**3)                           )
    return B

calculate_B_field(np.array([1,2,3]),np.array([1,2,3]))

array([3.81801774e-09, 7.63603548e-09, 1.14540532e-08])

## I will continue to update the file...

In [ ]:
import numpy as np
arr1=np.array([1,2,3])
arr2=np.array([6,7,8])


[ 6 14 24]


In [7]:
arr=[]
res=4
#One liner----
s_coords=[[[i,j,0] for j in range(res)] for i in range(res)]
print(s_coords)

[[[0, 0, 0], [0, 1, 0], [0, 2, 0], [0, 3, 0]], [[1, 0, 0], [1, 1, 0], [1, 2, 0], [1, 3, 0]], [[2, 0, 0], [2, 1, 0], [2, 2, 0], [2, 3, 0]], [[3, 0, 0], [3, 1, 0], [3, 2, 0], [3, 3, 0]]]


In [10]:
s_coords=[[[-((10)/2)+i*(10/(10-1)),((10)/2)-j*((10)/(10-1))    ,0] for j in range(10)] for i in range(10)]
print(s_coords)

[[[-5.0, 5.0, 0], [-5.0, 3.888888888888889, 0], [-5.0, 2.7777777777777777, 0], [-5.0, 1.6666666666666665, 0], [-5.0, 0.5555555555555554, 0], [-5.0, -0.5555555555555554, 0], [-5.0, -1.666666666666667, 0], [-5.0, -2.7777777777777786, 0], [-5.0, -3.8888888888888893, 0], [-5.0, -5.0, 0]], [[-3.888888888888889, 5.0, 0], [-3.888888888888889, 3.888888888888889, 0], [-3.888888888888889, 2.7777777777777777, 0], [-3.888888888888889, 1.6666666666666665, 0], [-3.888888888888889, 0.5555555555555554, 0], [-3.888888888888889, -0.5555555555555554, 0], [-3.888888888888889, -1.666666666666667, 0], [-3.888888888888889, -2.7777777777777786, 0], [-3.888888888888889, -3.8888888888888893, 0], [-3.888888888888889, -5.0, 0]], [[-2.7777777777777777, 5.0, 0], [-2.7777777777777777, 3.888888888888889, 0], [-2.7777777777777777, 2.7777777777777777, 0], [-2.7777777777777777, 1.6666666666666665, 0], [-2.7777777777777777, 0.5555555555555554, 0], [-2.7777777777777777, -0.5555555555555554, 0], [-2.7777777777777777, -1.66